# Trabalho 1

## Regressão Logística

### Ajuste da taxa de aprendizado e número de iterações

A primeira etada é ajustar a taxa de aprendizado e o número de iterações. Após alterar os outros hyperparameters (fator de regularização e grau dos atributos polinimiais) será necessário realizar o ajuste fino, mas será algum valor próximo ao encontrado nesta etapa.

** imports necessários: **

In [1]:
from regressao_logistica_regularizado import RegularizedLogisticRegression
from experimentos import Dataset as DATASET
from experimentos import ModelSelection
from experimentos import Plot
from dataset import Dataset
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

** varia alpha e número de iterações e plota os gráficos: **

In [ ]:
RLR = RegularizedLogisticRegression()
MS = ModelSelection()
DTS = DATASET()
PLT = Plot()

# carrega dataset
X,Y = RLR.load_dataset('datasets/mnist.csv', header=True)

# normaliza
X_ =  DTS.dataset_scaling(X)
alphas = [12, 11, 10.75, 10.5, 10.25, 10]
epochs = [1000]
Lambda = 0
colors = ['red', 'blue', 'pink', 'gray', 'yellow', 'green', 'black']
fold = 1
errors = {}
val_errors = {}
loss = {}
for train,val in MS.k_fold(X_, k=5, shuffle=True):
    fig, ax = plt.subplots()
    errors[str(fold)] = []
    val_errors[str(fold)] = []
    loss[str(fold)] = []
    for e in epochs:
        legends = []
        for i,a in enumerate(alphas):
            # ajusta o modelo
            RLR.fit(X_[train], Y[train], X_[val], Y[val], epochs=e, learning_rate=a, Lambda=Lambda, print_results=False)
            
            # calcula o erro no treino
            errors[str(fold)].append(RLR.train_error)
            # calcula o erro no teste
            val_errors[str(fold)].append(RLR.val_error)
            # salva loss
            loss[str(fold)].append(RLR.loss)
            
            ax.plot(range(e+1), RLR.loss, color=colors[i], linewidth=1)
            legends.append(mpatches.Patch(color=colors[i], label='alpha = ' + str(a)))
        #title = u'Ajuste da taxa com #epocas = ' + str(e) + ' e $\lambda$ = ' + str(Lambda) + ' (fold ' + str(fold) + ')'
        plt.ylim([0.1, 1])
        plt.xlabel(u'Épocas')
        plt.ylabel(u'Custo')
        
        box = ax.get_position()
        ax.set_position([box.x0, box.y0 + box.height * 0.1,
        box.width, box.height * 0.9])
        ax.legend(handles=legends[:], loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=True, ncol=5)
        #plt.title(title)
        plt.grid(True)
    
        fig.savefig('fold' + str(fold) + '.eps')
        plt.close(fig)
        
        #plt.show()
        
        fold += 1
        
error = []        
val_error = []
loss_ = []
for k in ['1', '2', '3', '4','5']:
    error.append(errors[k])
    val_error.append(val_errors[k])
    loss_.append(loss[k])
    
fig, ax = plt.subplots()
for i,a in enumerate(alphas):
    mean = np.asarray(loss_).mean(axis=0)[i]
    stdeviation = np.asarray(loss_).std(axis=0)[i]
    print a, mean[-1], stdeviation[-1]
    
    ax.plot(range(epochs[0]+1), mean, color=colors[i], linewidth=1)
    ax.fill_between(range(epochs[0]+1), mean-stdeviation, mean+stdeviation , color=colors[i], linewidth=1, alpha=0.2)
    
title = u'Curva de aprendizado média com #epocas = ' + str(e) + ' e $\lambda$ = ' + str(Lambda)
plt.ylim([0.1, 1])
plt.xlabel(u'Épocas')
plt.ylabel(u'Custo')

box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.1,
box.width, box.height * 0.9])
lgd = ax.legend(handles=legends[:], loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=True, ncol=5)
#plt.title(title)
plt.grid(True)

fig.savefig('media_std.png', bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

** erro por época ** (pode ignorar)

In [ ]:
for i in range(len(alphas)):
    plt.plot(range(epochs[0]+1), np.asarray(error).mean(axis=0)[i], color=colors[i], linewidth=1)
    #legends.append(mpatches.Patch(color=colors[i], label='alpha = ' + str(a)))
title = 'erro no treino'
plt.ylim([0.05, 0.6])
plt.xlabel('epocas')
plt.ylabel('erro')
plt.legend(handles=legends[:], loc='upper right')
plt.title(title)
plt.grid(True)

plt.savefig('erro_treino.eps')
plt.show()


In [ ]:
for i in range(len(alphas)):
    plt.plot(range(epochs[0]+1), np.asarray(val_error).mean(axis=0)[i], color=colors[i], linewidth=1)
    #legends.append(mpatches.Patch(color=colors[i], label='alpha = ' + str(a)))
title = 'erro na validacao'
plt.ylim([0.05, 0.6])
plt.xlabel('epocas')
plt.ylabel('erro')
plt.legend(handles=legends[:], loc='upper right')
plt.title(title)
plt.grid(True)

plt.savefig('erro_validacao.eps')
plt.show()

In [ ]:
# cada linha é um alpha, cada coluna uma época e cada valor é um erro binário médio dos k-folds
#print "erro medio no treino ", np.asarray(error).mean(axis=0).shape
#print "desvio padrao no treino ", np.asarray(error).std(axis=0)
#print "erro medio na validacao ", np.asarray(val_error).mean(axis=0).shape
#print "desvio padrao na validacao ", np.asarray(val_error).std(axis=0)

for i,a in enumerate(alphas):
    print "alpha = ", a
    print "erro medio final no treino ", np.asarray(error).mean(axis=0)[i][-1]
    print "desvio padrao final no treino ", np.asarray(error).std(axis=0)[i][-1]
    print "erro medio final na validacao ", np.asarray(val_error).mean(axis=0)[i][-1]
    print "desvio padrao final na validacao ", np.asarray(val_error).std(axis=0)[i][-1]
    print "\n"

### Curva de validação: atributos polinomiais

O objetivo deste experimento é analisar se a geração de atributos polinomiais trazem melhoria de
desempenho e, mais especificamente, qual o grau de polinômio mais adequado para a tarefa. Portanto,
deve-se variar o grau dos atributos polinomiais a partir de 1 (sem atributos polinomiais) até um valor que
a curva de validação indique claramente que há overfitting. Neste experimento, não utilize
regularização.

In [4]:
RLR = RegularizedLogisticRegression()
MS = ModelSelection()
DTS = DATASET()
PLT = Plot()

# carrega dataset
X,Y = RLR.load_dataset('datasets/mnist.csv', header=True)

degrees = range(1,10)
e = 1000

for d in degrees:
    X_pol = DTS.generate_polynomial_attributes(X, d)
    # normaliza
    X_pol =  DTS.dataset_scaling(X_pol)
    print X_pol.shape
    fold = 1
    errors = {}
    val_errors = {}
    loss = {}
    for train,val in MS.k_fold(X_pol, k=5, shuffle=True):
        errors[str(fold)] = []
        val_errors[str(fold)] = []
        loss[str(fold)] = []
        legends = []
        
        # ajusta o modelo
        RLR.fit(X_pol[train], Y[train], X_pol[val], Y[val], epochs=e, learning_rate=10, Lambda=0, print_results=False)

        # calcula o erro no treino
        errors[str(fold)].append(RLR.train_error[-1])
        # calcula o erro no teste
        val_errors[str(fold)].append(RLR.val_error[-1])
        # salva loss
        loss[str(fold)].append(RLR.loss)
        #print "erro no treino", RLR.train_error[-1]
        #print "erro na validacao", RLR.val_error[-1]
        fold += 1
        
    train_error = []        
    val_error = []
    loss_ = []
    for k in ['1', '2', '3', '4','5']:
        train_error.append(errors[k])
        val_error.append(val_errors[k])
        loss_.append(loss[k])

    print "grau dos atributos polinomiais: ", d
    print "erro médio no treino: ", np.asarray(train_error).mean()
    print "erro médio na validação: ", np.asarray(val_error).mean()
    print "\n"

(200, 4)
grau dos atributos polinomiais:  1
erro médio no treino:  0.08375
erro médio na validação:  0.095


(200, 14)
grau dos atributos polinomiais:  2
erro médio no treino:  0.0875
erro médio na validação:  0.095


(200, 34)
grau dos atributos polinomiais:  3
erro médio no treino:  0.085
erro médio na validação:  0.1


(200, 69)
grau dos atributos polinomiais:  4
erro médio no treino:  0.0825
erro médio na validação:  0.1


(200, 125)
grau dos atributos polinomiais:  5
erro médio no treino:  0.085
erro médio na validação:  0.095


(200, 209)
grau dos atributos polinomiais:  6
erro médio no treino:  0.07625
erro médio na validação:  0.105


(200, 329)
grau dos atributos polinomiais:  7
erro médio no treino:  0.0775
erro médio na validação:  0.09


(200, 494)
grau dos atributos polinomiais:  8
erro médio no treino:  0.0725
erro médio na validação:  0.095


(200, 714)
grau dos atributos polinomiais:  9
erro médio no treino:  0.07875
erro médio na validação:  0.105


